In [1]:
import math
import random
import numpy as np
import pandas as pd
from scipy.special import gamma
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
import itertools
from matplotlib import pyplot as plt
import torch as t

def factorial(n):
    if n == 0:
        return 1
    else:
        return n*factorial(n-1)

def normalize(a, ax=0):
    m = np.min(a,axis = ax)
    M = np.max(a,axis = ax)
    return (a-m)/(M-m)

import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(suppress = True)

data = pd.read_csv('data.csv').values
data = np.array(data)
np.random.shuffle(data)

len(data)


60000

In [2]:
rand = np.arange(60000)
np.random.shuffle(rand)
train_no = rand[:50000]

X = data

val_no = np.setdiff1d(rand,train_no)

X_train,X_val = X[train_no,:-1],X[val_no,:-1]
Y_train,Y_val = X[train_no,-1:],X[val_no,-1:]
Y_train.shape = (Y_train.shape[0],)  
Y_val.shape = (Y_val.shape[0],)
X_train.shape = (X_train.shape[0],X_train.shape[1])
X_val.shape = (X_val.shape[0],X_val.shape[1])

In [3]:
def changey(y, p):
    r = np.random.binomial(1,1-p,y.shape)
    x = np.random.randint(0,10,y.shape)
    return y*r+x-x*r

def changex(x, p):
    return x+np.random.normal(0,p,x.shape)

In [4]:
xtrain = []
ytrain = []

subsize = 1000

i=0
xtrain.append(X_train[i*subsize:(i+1)*subsize] )
ytrain.append(Y_train[i*subsize:(i+1)*subsize])
i+=1
xtrain.append(changex(X_train[i*subsize:(i+1)*subsize], 0.5) )
ytrain.append(Y_train[i*subsize:(i+1)*subsize])
i+=1
xtrain.append(changex(X_train[i*subsize:(i+1)*subsize], 1.0) )
ytrain.append(Y_train[i*subsize:(i+1)*subsize])
i+=1
xtrain.append(X_train[i*subsize:(i+1)*subsize] )
ytrain.append(changey(Y_train[i*subsize:(i+1)*subsize],1/10))
i+=1
xtrain.append(changex(X_train[i*subsize:(i+1)*subsize], 0.5) )
ytrain.append(changey(Y_train[i*subsize:(i+1)*subsize],1/10))
i+=1
xtrain.append(changex(X_train[i*subsize:(i+1)*subsize], 1.0) )
ytrain.append(changey(Y_train[i*subsize:(i+1)*subsize],1/10))


In [7]:
xtrain = np.array(xtrain).reshape(subsize * 6, 4)
ytrain = np.array(ytrain).reshape(subsize * 6, 1)

In [8]:
sub_train_size = 500

sample_size = 5000

new_ytrain = []
new_xtrain = []
    
    
train_data_n = np.concatenate((xtrain,ytrain),1)

for j in range(sample_size):
    if j % 100 == 0:
        print(j, end = ' ')
    copy = np.copy(train_data_n)
    np.random.shuffle(copy)
    
    sub_train_data = copy[:sub_train_size]

    sub_x_train = sub_train_data[:,:-1]
    sub_y_train = sub_train_data[:,-1:]

    sub_y_train.shape = (sub_train_size,)

    clf = LogisticRegression()
    clf.fit(sub_x_train, sub_y_train)
    predictions = clf.predict(X_val)

    accuracy = np.mean(predictions == Y_val)
    
    new_ytrain.append(accuracy)
    new_xtrain.append(copy[:sub_train_size])

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 

In [9]:
t.save(t.tensor(new_xtrain),'new_xtrain.pt')
t.save(t.tensor(new_ytrain),'new_ytrain.pt')

In [2]:
data

array([[0.01996664, 0.07502837, 0.03870484, 0.58489347, 0.        ],
       [0.86254881, 0.30053537, 0.50914283, 0.46915251, 1.        ],
       [0.16700914, 0.99209517, 0.40704909, 0.55397992, 1.        ],
       ...,
       [0.48621389, 0.97540449, 0.6886619 , 0.52392252, 1.        ],
       [0.32812737, 0.08335982, 0.16538633, 0.60135798, 0.        ],
       [0.42405844, 1.08734954, 0.67904326, 0.66858497, 1.        ]])

In [29]:
rand = np.arange(60000)
np.random.shuffle(rand)
train_no = rand[:50000]

X = data

val_no = np.setdiff1d(rand,train_no)

X_train,X_val = X[train_no,:-1],X[val_no,:-1]
Y_train,Y_val = X[train_no,-1:],X[val_no,-1:]
Y_train.shape = (Y_train.shape[0],)  
Y_val.shape = (Y_val.shape[0],)
X_train.shape = (X_train.shape[0],X_train.shape[1])
X_val.shape = (X_val.shape[0],X_val.shape[1])

In [30]:
clf = LogisticRegression()
clf.fit(X_train, Y_train)
predictions = clf.predict(X_val)

In [31]:
predictions

array([1., 0., 0., ..., 1., 0., 0.])

In [32]:
Y_val==predictions

array([ True,  True,  True, ...,  True,  True,  True])

In [33]:
np.mean(Y_val==predictions)

0.9865